In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch_smodel import *

# 1. Cargamos los datos de input y el target

In [2]:
df_input = pd.read_csv('df_input.csv', index_col=0)
grid_incendio = np.load('grid_incendio.npy')

In [3]:
df_input

,Theta,Rho,Temperature,Humidity
14:00:00,5.829400,0.425,35.0,28
14:30:00,5.497787,0.500,35.0,28
15:00:00,5.497787,0.625,35.5,28
15:30:00,5.619960,0.550,35.0,26
16:00:00,5.497787,0.500,35.0,25
16:30:00,5.654867,0.500,35.5,25
17:00:00,5.497787,0.500,36.0,24
17:30:00,5.497787,0.425,36.7,24
18:00:00,5.829400,0.375,36.0,25
18:30:00,2.129302,0.250,35.0,35


In [4]:
last_layer = torch.Tensor(grid_incendio[:, :, -1]) 
target_s = (last_layer==0)*1.
target_d = (last_layer==2)*1.

# 2. Creamos nuestro GRID AI

In [5]:
grid = AI_Grid(N=31, K=17)
grid.__param__(inc=1, partition=[0.1, 0.5, 0.9])
grid.load_data(data=df_input)
grid.AI_MonteCarlo(input_size=2, target_s=last_layer)

Number epoch:  0


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
grid.p0, grid.div

(tensor([0.2907, 0.2907, 0.2926, 0.3165, 0.3298, 0.3318, 0.3475, 0.3504, 0.3338,
         0.2110, 0.1210, 0.0995, 0.0963, 0.0956, 0.0897, 0.0889, 0.0834],
        dtype=torch.float64, grad_fn=<ToCopyBackward0>),
 tensor([2.6704, 2.6704, 2.6562, 2.4494, 2.3389, 2.3248, 2.2002, 2.1804, 2.3106,
         3.4437, 4.6333, 5.0213, 5.0779, 5.0920, 5.2167, 5.2308, 5.3554],
        dtype=torch.float64, grad_fn=<ToCopyBackward0>))

In [ ]:
grid.X0.shape

torch.Size([31, 31, 18])